In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from tobler.util import h3fy

from DataAnalysis import DataPreparation as DataPreparation
from DataAnalysis import DataMethodPreparation as DataMethodPreparation
from DataAnalysis import DataDonwload as DataDonwload

from AssastmentMethod import AspectLaw as AspectLaw
from AssastmentMethod import AspectPhysical as AspectPhysical
from AssastmentMethod import AspectSpatial as AspectSpatial
from AssastmentMethod import AspectEconomic as AspectEconomic
from AssastmentMethod import General as General

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Загрузка исходного файла с ОКН
heritage_origin = pd.read_csv('Example_files/All_heritage.csv')

# Выборка объектов в нужном регионе
REGION = 'Вологодская область'
ch_region, ch_region_coor, ch_region_none = DataPreparation.separator_reestr(heritage_origin, REGION) # Общий датасет, ГДФ(что можно найти), ДФ без координат оставшиеся

In [ ]:
# Загрузка отчета о кадастровой оценке
report = gpd.read_file('Example_files/buildings_report.geojson') 
report = DataPreparation.number_report(report)

In [4]:
# Выборка необходимых столбцов для анализа
ch_region = ch_region[['Номер в реестре', 'учетный номер', 'Объект', 'Регион', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'описание предмета охраны', 'Принадлежность к Юнеско', 'наименование документа', 'DATE_BUILD', 'lat', 'lon', 'geometry']]
report = report[['Номер в реестре','учетный номер', 'Кадастровый номер', 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН', 'Кадастровые номера помещений в объекте недвижимости', 'Площадь, кв.м', 'Назначение здания', 'Год завершения строительства', 'Количество этажей (в том числе подземных)', 'Количество подземных этажей', 'Материал наружных стен', 'Класс конструктивной системы', 'Группа капитальности', 'Объект признан аварийным ', 'Функциональная группа', 'Подгруппа', 'usage', 'area', 'cad_cost', 'geometry']]

# Преобразование столбцов в нужный формат
ch_region['Номер в реестре'] = ch_region['Номер в реестре'].astype(str)
ch_region['учетный номер'] = ch_region['учетный номер'].astype(str)

In [5]:
# Разделение в зависимости от наличие информации
report_1 = report[report['Номер в реестре'] != 0]
report_2 = report[report['Номер в реестре'] == 0]

# объединяем и удалаяем лишние стобцы
gdf_merged_1 = pd.merge(report_1, ch_region, on='Номер в реестре', how='inner')
gdf_merged_2 = pd.merge(report_2, ch_region, on='учетный номер', how='inner')

gdf_merged = pd.concat([gdf_merged_1, gdf_merged_2], ignore_index=True)
gdf_merged = gdf_merged.drop(['Номер в реестре_x', 'Номер в реестре_y', 'учетный номер_y', 'учетный номер_x'], axis = 1)

In [ ]:
for i in range(len(gdf_merged)):
    if gdf_merged.loc[i, 'geometry_y'] != None:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_y'].centroid # Если геометрия не пустая, то из полигона делаем точку
    else:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_x']
gdf_merged.rename(columns={'geometry_y':'geometry'}, inplace = True)
gdf_merged = gpd.GeoDataFrame(gdf_merged,geometry='geometry', crs='EPSG:4326')
gdf_merged['geometry']=gdf_merged['geometry'].centroid

In [7]:
# добавляекм кадастровую стоимость
cad_cost = pd.read_excel('Example_files/report_cost_1.xlsx')
cad_cost = cad_cost[['Кадастровый номер','Аварийность объекта','Экономическое устаревание', 'Эффективный возраст', 'Срок экономической жизни, лет', 'Физический износ', 'Совокупный износ','Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м', 'Кадастровая стоимость (при оценке), руб']]
gdf_merged = pd.merge(gdf_merged, cad_cost, how='left', on='Кадастровый номер')
gdf_merged = gdf_merged.drop('geometry_x', axis=1)

In [8]:
# Переименуем столбцы для приведение к требуемому виду
gdf_merged.rename(columns={'Полный адрес': 'ADDRESS', 
                                 'Объект': 'NAME', 
                                 'Номер в реестре': 'REG_NUMBER', 
                                 'Категория историко-культурного значения': 'HIST_CAT',
                                 'Назначение здания': 'OCH_USE', 
                                 'Кадастровый номер': 'KADASTROKS', 
                                 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН': 'KADASTRZY', 
                                 'Совокупный износ': 'WEAR_PRCNT', 
                                 'Аварийность объекта': 'PR_ALARM', 
                                 'Материал наружных стен': 'MATERIAL', 
                                 'описание предмета охраны': 'PROTECTION', 
                                 'Принадлежность к Юнеско': 'UNESCO_TYPE', 
                                 'usage': 'ZY_VRI',
                                 'Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м': 'BUILD_KADCOST', 
                                 'cad_cost': 'ZY_KADCOST', 
                                 'area': 'ZY_AREA', 
                                 'Площадь, кв.м': 'BLD_AREA'                                 
                                 }, inplace=True)

In [9]:
# готовим ДФ для последующего анализа
gdf_merged = DataMethodPreparation.start_all(gdf_merged)

In [10]:
gdf_merged['ARCH_STYLE'] = ''
gdf_merged['BLD_ACUSE'] = 1

In [11]:
heritage_merged = gdf_merged[['NAME','ADDRESS', 'REG_NUMBER', 'HIST_CAT','OCH_USE', 'BLD_ACUSE', 'KADASTROKS', 'KADASTRZY', 'FLOOR', 'DATE_BUILD', 'BLD_AREA', 'ZY_AREA', 'ZY_KADCOST', 'BUILD_KADCOST', 'BLD_USAGE', 'ZY_VRI', 'PROTECTION', 'PROTECTION_SUM', 'UNESCO_TYPE', 'ARCH_STYLE', 'MATERIAL', 'PR_ALARM', 'WEAR_PRCNT', 'geometry']]

In [12]:
# Скачиваем данные из OSM по двум исследуемым городам
region_V, building_osm_V, shop_V, cafe_V, public_transport_V = DataDonwload.data_osm('город Вологда')

In [2]:
region_T, building_osm_T, shop_T, cafe_T, public_transport_T= DataDonwload.data_osm('город Тотьма')

Не найдены остановки общественного транспорта. Проверьте правильность написания названия территории или внесите данные вручную


In [ ]:
# Выбираем объекты расположенные в городе Вологда
heritage_vologda = gpd.overlay(heritage_merged, region_V, how='intersection')
# Выбираем объекты расположенные в городе Тотьма
heritage_totma = gpd.overlay(heritage_merged, region_T, how='intersection')

In [15]:
# Дополнительная информация из викимапии
wiki_gdf = pd.read_csv('Example_files/file_from_wiki_vologda.csv')
wiki_gdf = gpd.GeoDataFrame(wiki_gdf, geometry=gpd.points_from_xy(wiki_gdf.lon, wiki_gdf.lat), crs='EPSG:4326')

wiki_gdf = wiki_gdf.to_crs('EPSG:32637')
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')

heritage_vologda['geometry'] = heritage_vologda['geometry'].buffer(20)
heritage_vologda['ARCH_STYLE']=''

for i in range(len(heritage_vologda)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(heritage_vologda.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        heritage_vologda.loc[i, 'ARCH_STYLE'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

for i in range(len(building_osm_V)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(building_osm_V.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        building_osm_V.loc[i, 'ARCH_STYLE'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

In [18]:
heritage_vologda['geometry'] = heritage_vologda['geometry'].centroid

In [28]:
heritage_totma.to_file('totma_wip.geojson')

In [19]:
heritage_vologda.to_file('vologda_wip.geojson')

In [20]:
heritage_vologda = gpd.read_file('vologda_wip.geojson')

In [ ]:
heritage_vologda = AspectPhysical.start_all(heritage_vologda)
heritage_vologda = AspectLaw.start_all(heritage_vologda)
heritage_vologda = AspectEconomic.start_all(heritage_vologda)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\AssastmentMethod.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf['SCORE_ZY_VRI'].fillna(0, inplace = True)


In [ ]:
heritage_vologda_G = AspectSpatial.graph_creator(heritage_vologda, region_V)
heritage_vologda_iso = AspectSpatial.iso_create(heritage_vologda, heritage_vologda_G)
heritage_vologda_vision = AspectSpatial.vision(heritage_vologda, building_osm_V, building_osm_V)

In [ ]:
heritage_totma_G = AspectSpatial.graph_creator(heritage_totma, region_T)
heritage_totma_iso = AspectSpatial.iso_create(heritage_totma, heritage_totma_G)
heritage_totma_vision = AspectSpatial.vision(heritage_totma, building_osm_T, building_osm_T)

In [37]:
# Сохранение изохронов и зон видимости в файлы
heritage_totma_iso.to_file('heritage_totma_iso.geojson')
heritage_totma_vision.to_file('heritage_totma_vision.geojson')

In [3]:
# Загрузка изохронов и зон видимости в файлы
heritage_totma = gpd.read_file('totma_wip.geojson')
heritage_totma_iso = gpd.read_file('H:/Documents/Python/Risks_of_investing_in_cultural_heritage/output_files/heritage_totma_iso.geojson')
heritage_totma_vision = gpd.read_file('H:/Documents/Python/Risks_of_investing_in_cultural_heritage/output_files/heritage_totma_vision.geojson')
public_transport_T = gpd.read_file('H:/Documents/Python/Risks_of_investing_in_cultural_heritage/output_files/heritage_totma_vision.geojson')

In [4]:
heritage_totma = AspectPhysical.start_all(heritage_totma)
heritage_totma = AspectLaw.start_all(heritage_totma)
heritage_totma = AspectEconomic.start_all(heritage_totma)
heritage_totma  = AspectSpatial.start_all(heritage_totma, region_T, shop_T, cafe_T, public_transport_T, building_osm_T, heritage_totma_iso, heritage_totma_vision)
heritage_totma = General.calculate_scores(heritage_totma)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\AssastmentMethod.py:215: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf.loc[i,'SCORE_SERVICES'] += 0.5


In [7]:
heritage_totma.to_file('heritage_totma.geojson')
heritage_totma.to_excel('heritage_totma.xlsx')

In [ ]:
heritage_vologda['geometry'] = heritage_vologda['geometry'].centroid

In [ ]:
# Сохранение файла
heritage_vologda.to_file('Example_files/volscenario.geojson')

In [ ]:
heritage_totma['score_land_use'] = 0
heritage_totma['arch_style'] = ''

In [ ]:
heritage_totma = AspectPhysical.start_all(heritage_totma)
heritage_totma = AspectLaw.start_all(heritage_totma)
heritage_totma = AspectEconomic.start_all(heritage_totma)
heritage_totma = AspectSpatial.start_all(heritage_totma, shop_t, cafe_t, public_transport_t,building_osm_t, hex_totma)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\AssastmentMethod.py:258: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf.loc[i, 'score_historicity'] += 0.1


In [ ]:
heritage_totma = General.calculate_scores(heritage_totma)
heritage_totma = General.risks_assastment(heritage_totma)
heritage_totma['geometry'] = heritage_totma['geometry'].centroid

In [ ]:
heritage_totma

,Номер в реестре,Кадастровый номер,"Кадастровые номера ЗУ, в пределах которых расположен данный ОН",Кадастровые номера помещений в объекте недвижимости,"Площадь, кв.м",Назначение здания,Год завершения строительства,Количество этажей (в том числе подземных),Количество подземных этажей,Материал наружных стен,...,count_public_transport,score_services,score_transport,score_historicity,total_spatial_score,total_spatial_percent,total_score,total_percent,risks,risks_num
0,351710758560005,35:14:0705022:51,35:14:0705022:25,None,441.7,Нежилое,2008.0,2,None,Рубленые,...,0.0,0.5,0,0.4,1.19,29.75,11.524661,57.00,10001;10002;10004;,3.0
1,351710758510035,35:14:0705013:49,35:14:0705013:11,None,165.2,Нежилое,2001.0,2,None,Кирпичные,...,0.0,1.0,0,0.4,1.94,48.50,12.779585,63.20,10001;10002;,2.0
2,351520257260006,35:14:0703001:35,35:14:0703001:221,None,301.3,Нежилое,1870.0,1,None,Кирпичные,...,0.0,0.0,0,0.4,0.44,11.00,11.082691,54.81,10001;10002;,2.0
3,351520257260006,35:14:0703001:32,35:14:0703001:221,None,825.4,Нежилое,1689.0,2,None,Кирпичные,...,0.0,0.0,0,0.1,0.11,2.75,10.752692,53.18,10001;10002;,2.0
4,351520257260006,35:14:0703001:30,35:14:0703001:221,None,1175.0,Нежилое,1796.0,3,1,Кирпичные,...,0.0,0.0,0,0.1,0.11,2.75,10.752692,53.18,10001;10002;,2.0
5,351520257260006,35:14:0703001:33,35:14:0703001:221,None,1583.7,Нежилое,1800.0,3,None,Кирпичные,...,0.0,0.0,0,0.1,0.11,2.75,10.752692,53.18,10001;10002;,2.0
6,351520257260006,35:14:0703001:36,35:14:0703001:224,None,47.5,Нежилое,NaN,3,1,Кирпичные,...,0.0,0.0,0,0.1,0.11,2.75,11.715653,57.94,10001;10002;,2.0
7,351520257260006,35:14:0703001:34,35:14:0703001:222,None,532.6,Нежилое,NaN,1,None,Кирпичные,...,0.0,0.0,0,0.4,0.44,11.00,11.783152,58.27,10001;10002;30002;,3.0
8,351610758500006,35:14:0704021:17,35:14:0704021:5,None,385.1,Нежилое,1745.0,2,None,Кирпичные,...,1.0,0.5,0,0.4,1.19,29.75,11.072691,54.76,10001;10002;30001;,3.0
9,351710748050005,35:14:0704023:37,35:14:0704023:185,None,396.0,Нежилое,1917.0,2,None,Кирпичные,...,1.0,0.5,0,0.4,1.19,29.75,13.070191,64.64,10001;10002;,2.0


In [ ]:
hex_totma = AspectSpatial.merge_hex_services(hex_totma, cafe_t, shop_t, public_transport_t)

In [ ]:
heritage_totma = AspectSpatial.services_hex(heritage_totma, hex_totma)

In [ ]:
heritage_totma = AspectSpatial.score_spatial(heritage_totma)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\AssastmentMethod.py:272: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf.loc[i,'score_services'] += 0.5


In [ ]:
heritage_totma['total_spatial_score'] = 1.50 * heritage_totma['score_services'] + 1.40 * heritage_totma['score_transport'] + 1.10 * heritage_totma['score_historicity']

In [ ]:
heritage_totma['total_spatial_percent'] = round(heritage_totma['total_spatial_score'] / 4 * 100, 2)

In [ ]:
# Сохранение файла
heritage_totma.to_file('Example_files/totma_scenario_3.geojson')

In [ ]:
hex_vologda = AspectSpatial.merge_hex_services(hex_vologda, cafe, shop, public_transport)
heritage_vologda = AspectSpatial.services_hex(heritage_vologda, hex_vologda)
heritage_vologda = AspectSpatial.score_spatial(heritage_vologda)
heritage_vologda['total_spatial_score'] = 1.50 * heritage_vologda['score_services'] + 1.40 * heritage_vologda['score_transport'] + 1.10 * heritage_vologda['score_historicity']
heritage_vologda['total_spatial_percent'] = round(heritage_vologda['total_spatial_score'] / 4 * 100, 2)


h:\Documents\Python\Risks_of_investing_in_cultural_heritage\AssastmentMethod.py:272: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf.loc[i,'score_services'] += 0.5


In [ ]:
# Сохранение файла
heritage_vologda.to_file('Example_files/vologda_scenario_1.geojson')

In [ ]:
heritage_vologda.loc[[3,93,71], 'total_physical_score'] = 1.19 * heritage_vologda.loc[[3,93,71],'score_deterioration'] + 1.05 * heritage_vologda.loc[[3,93,71],'score_material'] + 1.10 * heritage_vologda.loc[[3,93,71],'score_function'] + 1.29 * heritage_vologda.loc[[3,93,71],'score_percentage_of_construction']  + 1.05 * heritage_vologda.loc[[3,93,71],'score_accident'] + 1.19 * heritage_vologda.loc[[3,93,71],'score_engineering_communications']
heritage_vologda['total_physical_percent'] = round(heritage_vologda['total_physical_score'] / 8 * 100, 2)

In [ ]:
heritage_vologda_2 = heritage_vologda.copy()
heritage_totma_2 = heritage_totma.copy()

In [ ]:
# Симуляция сценариев 2 и 4
heritage_vologda_2['score_engineering_communications'] = 0
heritage_totma_2['score_engineering_communications'] = 0


heritage_vologda_2['score_support_measures'] = 0
heritage_totma_2['score_support_measures'] = 0


heritage_vologda_2['total_economic_score'] = 1.22 * heritage_vologda_2['score_cadstral_value_building'] + 1.22 * heritage_vologda_2['score_cadstral_value_land'] + 1.78 * heritage_vologda_2['score_support_measures']
heritage_vologda_2['total_economic_percent'] = round(heritage_vologda_2['total_economic_score'] / 4.22 * 100, 2)

heritage_totma_2['total_economic_score'] = 1.22 * heritage_totma_2['score_cadstral_value_building'] + 1.22 * heritage_totma_2['score_cadstral_value_land'] + 1.78 * heritage_totma_2['score_support_measures']
heritage_totma_2['total_economic_percent'] = round(heritage_totma_2['total_economic_score'] / 4.22 * 100, 2)

heritage_totma_2['score_transport'] = 0
heritage_totma_2['total_spatial_score'] = 1.50 * heritage_totma_2['score_services'] + 1.40 * heritage_totma_2['score_transport'] + 1.10 * heritage_totma_2['score_historicity']
heritage_totma_2['total_spatial_percent'] = round(heritage_totma_2['total_spatial_score'] / 4 * 100, 2)

In [ ]:
def re_calculate_ph(gdf):
    gdf['total_physical_score'] = 1.19 * gdf['score_deterioration'] + 1.05 * gdf['score_material'] + 1.10 * gdf['score_function'] + 1.29 * gdf['score_percentage_of_construction']  + 1.05 * gdf['score_accident'] + 1.19 * gdf['score_engineering_communications'] + 1.14 # за текущее использование
    gdf['total_physical_percent'] = round(gdf['total_physical_score'] / 8 * 100, 2)
    return(gdf)

In [ ]:
def calculate_scores(gdf):
    gdf['total_score'] = gdf['total_physical_score'] + gdf['total_spatial_score'] + gdf['total_law_score'] + gdf['total_economic_score']
    gdf['total_percent'] = round(gdf['total_score'] / 20.22 * 100, 2)
    return(gdf)

In [ ]:
heritage_totma_2['score_transport'] = 0
heritage_totma_2['total_spatial_score'] = 1.50 * heritage_totma_2['score_services'] + 1.40 * heritage_totma_2['score_transport'] + 1.10 * heritage_totma_2['score_historicity']
heritage_totma_2['total_spatial_percent'] = round(heritage_totma_2['total_spatial_score'] / 4 * 100, 2)

In [ ]:
heritage_vologda_2 = re_calculate_ph(heritage_vologda_2)

In [ ]:
heritage_totma.loc[[1,9,10,11,13,14,16,17, 20],'score_transport']

1     1
9     1
10    1
11    1
13    1
14    1
16    1
17    1
20    1
Name: score_transport, dtype: int64

In [ ]:
heritage_totma['total_spatial_score'] = 1.50 * heritage_totma['score_services'] + 1.40 * heritage_totma['score_transport'] + 1.10 * heritage_totma['score_historicity']

In [ ]:
heritage_totma['total_spatial_score'].max()

3.34

In [ ]:
heritage_vologda = calculate_scores(heritage_vologda)
heritage_vologda = General.risks_assastment(heritage_vologda)

heritage_vologda_2 = calculate_scores(heritage_vologda_2)
heritage_vologda_2 = General.risks_assastment(heritage_vologda_2)

heritage_totma = calculate_scores(heritage_totma)
heritage_totma = General.risks_assastment(heritage_totma)

heritage_totma_2 = calculate_scores(heritage_totma_2)
heritage_totma_2 = General.risks_assastment(heritage_totma_2)

In [ ]:
381310014200006, 351410049600005, 351410053110006

In [ ]:
heritage_vologda.to_csv('Example_files/vologda_scenario_1.csv')
heritage_vologda_2.to_csv('Example_files/vologda_scenario_2.csv')
heritage_totma.to_csv('Example_files/totma_scenario_3.csv')
heritage_totma_2.to_csv('Example_files/totma_scenario_4.csv')

In [ ]:
heritage_vologda.to_file('Example_files/_vologda_scenario_1.geojson')
heritage_vologda_2.to_file('Example_files/_vologda_scenario_2.geojson')
heritage_totma.to_file('Example_files/_totma_scenario_3.geojson')
heritage_totma_2.to_file('Example_files/_totma_scenario_4.geojson')

In [ ]:
heritage_vologda_2['total_percent'].mean()

51.42304093567252

In [ ]:
heritage_totma['total_percent'].mean()

63.16571428571429

In [ ]:
heritage_vologda_2.to_file('Example_files/_vologda_scenario_2.geojson')

In [ ]:
region = ox.geocode_to_gdf(query="Вологодская область")

In [ ]:
region.to_file('Vologds.geojson')

In [ ]:
heritage_vologda['total_percent'].max()

83.05

In [ ]:
heritage_vologda[heritage_vologda['total_percent'] > 80]

,Номер в реестре,Кадастровый номер,"Кадастровые номера ЗУ, в пределах которых расположен данный ОН",Кадастровые номера помещений в объекте недвижимости,"Площадь, кв.м",Назначение здания,Год завершения строительства,Количество этажей (в том числе подземных),Количество подземных этажей,Материал наружных стен,...,count_public_transport,score_services,score_transport,score_historicity,total_spatial_score,total_spatial_percent,total_score,total_percent,risks,risks_num
26,351711304950005,35:24:0304010:161,35:24:0304010:92,None,549.9,Жилое,1910.0,2,None,Рубленые,...,2.0,1.0,1,0.4,3.34,83.50,16.487433,81.54,10001;10002;10004;,3.0
82,351711288070005,35:24:0202011:63,35:24:0202011:37,35:24:0202011:100; 35:24:0202011:101; 35:24:02...,2569.1,Нежилое,2007.0,5,1,Кирпичные,...,2.0,1.0,1,0.1,3.01,75.25,16.295905,80.59,,0.0
117,351410062350006,35:24:0201012:100,35:24:0201012:88,35:24:0201012:168; 35:24:0201012:169; 35:24:02...,308.0,Жилое,1893.0,2,None,Рубленые,...,2.0,1.0,1,0.4,3.34,83.50,16.469933,81.45,10001;10002;10004;,3.0
169,NaN,35:24:0201012:123,35:24:0201012:84,35:24:0201012:197; 35:24:0201012:198; 35:24:02...,358.2,Многоквартирный дом,1902.0,2,None,Рубленые,...,2.0,1.0,1,0.4,3.34,83.50,16.792433,83.05,10001;10002;10004;,3.0


In [ ]:
heritage_totma[heritage_totma['total_percent'] > 75]

,Номер в реестре,Кадастровый номер,"Кадастровые номера ЗУ, в пределах которых расположен данный ОН",Кадастровые номера помещений в объекте недвижимости,"Площадь, кв.м",Назначение здания,Год завершения строительства,Количество этажей (в том числе подземных),Количество подземных этажей,Материал наружных стен,...,count_public_transport,score_services,score_transport,score_historicity,total_spatial_score,total_spatial_percent,total_score,total_percent,risks,risks_num
16,351710745040005,35:14:0705019:47,35:14:0705019:101,35:14:0705019:97,141.2,Нежилое,1896.0,1,None,Кирпичные,...,1.0,1.0,1,0.4,3.34,48.5,15.220192,75.27,10001;10002;,2.0


In [ ]:
heritage_vologda['count']

0      4
1      3
2      0
3      4
4      0
      ..
167    2
168    0
169    0
170    0
171    0
Name: count, Length: 172, dtype: int64